In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import datetime

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

os.makedirs(timestamp)

In [3]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["TRAIN_SPLIT"] = 100000
param_list["BATCH_SIZE"] = 256
param_list["BUFFER_SIZE"] = 200000
param_list["EVALUATION_INTERVAL"] = 300
param_list["VAL_STEPS"] = 50

In [4]:
pattern_length = 4
data = np.genfromtxt("data/SEG_delta_pattern_length_{}.csv".format(pattern_length), delimiter="\n")
data

array([-1.58765354e+09,  6.17292586e+09, -6.18790972e+09, ...,
        4.75252240e+07, -7.52773980e+07,  3.22916640e+07])

In [11]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.4, shuffle=False, random_state=42)
train_set = train_set.reshape(-1, 1)
test_set = test_set.reshape(-1, 1)
train_set, test_set

(array([[-1.58765354e+09],
        [ 6.17292586e+09],
        [-6.18790972e+09],
        ...,
        [ 6.55360000e+04],
        [-1.66730240e+07],
        [-5.88800000e+03]]),
 array([[   878848.],
        [-11182192.],
        [-11128760.],
        ...,
        [ 47525224.],
        [-75277398.],
        [ 32291664.]]))

In [12]:
from sklearn.preprocessing import MinMaxScaler
import joblib

minmax_scaler = MinMaxScaler()

train_set_scaled = minmax_scaler.fit_transform(train_set)
joblib.dump(minmax_scaler, "{}/scaler.pkl".format(timestamp))
train_set_scaled

array([[0.39820164],
       [0.89577508],
       [0.10325391],
       ...,
       [0.49999905],
       [0.49892585],
       [0.49999447]])

In [19]:
pd.Series(data=train_set_scaled.reshape(1, -1)[0]).value_counts()

0.499996    1380
0.499995     574
0.499996     527
0.499995     387
0.499996     302
            ... 
0.494874       1
0.489484       1
0.486970       1
0.588957       1
0.000000       1
Length: 63732, dtype: int64

In [33]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [49]:
x_train, y_train = generate_timeseries(train_set_scaled, 0, param_list["TRAIN_SPLIT"], param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
x_val, y_val = generate_timeseries(train_set_scaled, param_list["TRAIN_SPLIT"], None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat()
train_data = train_data.cache().shuffle(param_list["BUFFER_SIZE"]).batch(param_list["BATCH_SIZE"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).repeat()
val_data = val_data.cache().batch(param_list["BATCH_SIZE"])

In [58]:
test_set_scaled = minmax_scaler.fit(test_set)

x_test, y_test = generate_timeseries(test_set_scaled, 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).repeat()
test_data = test_data.cache().batch(param_list["BATCH_SIZE"])

TypeError: object of type 'MinMaxScaler' has no len()

In [51]:
from tensorboard.plugins.hparams import api as hp

HP_NUM_LSTM_1_UNITS = hp.HParam("num_LSTM_1_units", hp.Discrete([32, 64, 128]))
HP_NUM_LSTM_2_UNITS = hp.HParam("num_LSTM_2_units", hp.Discrete([32, 64, 128]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.5))

METRIC_ACCURACY = 'mae'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_LSTM_1_UNITS, HP_NUM_LSTM_2_UNITS, HP_DROPOUT],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='mae')]
    )    

In [57]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(hparams[HP_NUM_LSTM_1_UNITS], return_sequences=True, input_shape=x_train.shape[-2:]),
        tf.keras.layers.LSTM(hparams[HP_NUM_LSTM_2_UNITS]),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(FUTURE_TARGET),
        tf.keras.layers.Activation("relu")
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(train_data, epochs=1, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VAL_STEPS, callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir),  # log metrics
        hp.KerasCallback(log_dir, hparams),  # log hparams
    ],)
    _, mae = model.evaluate(x_test, y_test)
    return model

In [53]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)